This notebook will be to configure the the ml-quigley workspace.

In [24]:
#import the azure core libraries
from azureml.core import Workspace
from azureml.core.experiment import Experiment
from azureml.core.run import Run

#used for configuyring rusn with compute targets
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute

#used to define dependencies on the compute instance
from azureml.core.conda_dependencies import CondaDependencies

#used to define and create the datasets
from azureml.core import Datastore
from azureml.core import Dataset

#for running the training scripts with experiments
from azureml.core import ScriptRunConfig


In [25]:
ws = Workspace.get(  name="ml-quigley"
                   , subscription_id='8d8bcbfe-86b4-42bb-a470-a8cf97c98d69'
                   , resource_group='project-quigley')

In [7]:
experiment = Experiment(
                          workspace=ws
                        , name='quigley-overall-classifier'
                        )

In [20]:
#list of vms available for compute
list_vms = AmlCompute.supported_vmsizes(workspace=ws)

In [22]:
for l in list_vms:
    if l['vCPUs'] > 4:
        print(l)

{'name': 'Standard_DS4_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 28.0, 'maxResourceVolumeMB': 57344}
{'name': 'Standard_DS5_v2', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB': 114688}
{'name': 'Standard_DS13_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB': 114688}
{'name': 'Standard_DS14_v2', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 112.0, 'maxResourceVolumeMB': 229376}
{'name': 'Standard_DS15_v2', 'vCPUs': 20, 'gpus': 0, 'memoryGB': 140.0, 'maxResourceVolumeMB': 286720}
{'name': 'Standard_D8s_v3', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 32.0, 'maxResourceVolumeMB': 65536}
{'name': 'Standard_D16s_v3', 'vCPUs': 16, 'gpus': 0, 'memoryGB': 64.0, 'maxResourceVolumeMB': 131072}
{'name': 'Standard_D32s_v3', 'vCPUs': 32, 'gpus': 0, 'memoryGB': 128.0, 'maxResourceVolumeMB': 262144}
{'name': 'Standard_D4_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 28.0, 'maxResourceVolumeMB': 409600}
{'name': 'Standard_D13_v2', 'vCPUs': 8, 'gpus': 0, 'memoryGB': 56.0, 'maxResourceVolumeMB'

In [ ]:
compute_config = RunConfiguration()
compute_config.target = "amlcompute"
compute_config.amlcompute.vm_size = "Standard_NV6"

In [ ]:
#create dependiencies for the compute target
dependencies = CondaDependencies()
dependencies.add_pip_package("tensorflow")
dependencies.add_pip_package("numpy")
dependencies.add_pip_package("pandas")
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("gensim")
dependencies.add_pip_package("keras")
dependencies.add_pip_package("tensorflow_hub")
compute_config.environment.python.conda_dependencies = dependencies

In [ ]:
#register the compute


In [ ]:
#get the compute


In [26]:
#define datastore
datastore = Datastore.get(ws, datastore_name='quigleydl')

In [19]:
#creates datasets from the files on the data store



datastore_paths = [(datastore,'prepped-files/leg-meta/*LegOverallData.csv')]
leg_meta_ds = Dataset.File.from_files(path=datastore_paths)

leg_meta_ds = leg_meta_ds.register(workspace=ws
                                   ,name='leg_meta_ds'
                                   ,description='legiscan meta data with events (bill_id by time_t)')

In [28]:
datastore_paths = [(datastore,'prepped-files/leg-text/*legtext.corpus')]
leg_text_ds = Dataset.File.from_files(path=datastore_paths)

leg_text_ds = leg_text_ds.register(workspace=ws
                                   ,name='leg_text_ds'
                                   ,description='legislative text data, cleaned with words separated into lists')

In [ ]:
#Tabular data set for the document clusters
datastore_paths = [(datastore,'prepped-files/leg-text-clusters/*.csv')]
leg_text_clusters_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

leg_text_clusters_ds = leg_text_clusters_ds.register(workspace=ws
                                   ,name='leg_text_clusters_ds'
                                   ,description='legislative texts clustered')

In [ ]:
#Tabular data set for the document embeddings
datastore_paths = [(datastore,'prepped-files/leg-text-embeddings/*.csv')]
leg_text_embeddings_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

leg_text_embeddings_ds = leg_text_clusters_ds.register(workspace=ws
                                   ,name='leg_text_embeddings_ds'
                                   ,description='legislative text embeddings')

In [ ]:
#tabular data set for the cumulative events compilation
datastore_paths = [(datastore,'prepped-files/leg-text-embeddings/*.csv')]
leg_text_embeddings_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

leg_text_embeddings_ds = leg_text_clusters_ds.register(workspace=ws
                                   ,name='leg_text_embeddings_ds'
                                   ,description='legislative text embeddings')
